In [1]:
import pandas as pd

from langchain_dartmouth.llms import ChatDartmouth

In [2]:
docs = pd.read_csv("data/raw/2024_COFE_SS_sample_text.csv")
question = docs.loc[0, "outcometxt"]
question

'Please use the space below to describe the most important outcomes of your time as an undergraduate.'

In [3]:
docs = docs.iloc[1:]  # The first row is a comment on the column contents
docs = docs.dropna(subset="outcometxt")
responses = docs["outcometxt"].to_list()
responses[:5]

['Research',
 'My most valuable experiences at Dartmouth had nothing to do with the classes that I was taking; if anything, those classes inhibited my from doing what I actually cared about, which was my extracurricular activities, political advocacy, and social connection. The environment that Dartmouth placed me in gave me the resources and situations necessary to excel at my non-classroom activities, which were the ones that I actually cared about since they will be what I do after I graduate and are what make me happy.',
 'i am confident that i will not have a stable life or income for at least a decade after graduation ',
 '- the connections I’ve made (whether with students , faculty or alumni)',
 'I feel like I was torn between two majors for a long time. Dartmouth helped me work through both majors and find a middle ground which I’m pursing in my Master’s']

In [4]:
llm = ChatDartmouth(
    model_name="llama-3-1-8b-instruct",
    temperature=0,
    seed=42,
    max_tokens=60_000,
)

example_response = (
    "Dartmouth has been a transformative experience that's shaped my academic, "
    "personal, and professional growth. I've gained invaluable knowledge and "
    "skills through rigorous courses in econ and international relations, which "
    "have helped me develop a nuanced perspective on global issues. However, I've "
    "also struggled with the intense academic pressure and lack of diversity in "
    "certain departments."
)

example_output = """{
    "response" : <original response>,
    "topics": [
        {"name": "academic and personal growth", "sentiment": "positive"},
        {"name": "academic rigor", "sentiment": "positive"},
        {"name": "academic pressure", "sentiment": "negative"},
        {"name": "lack of diversity", "sentiment": "negative"},
    ]
}"""

prompt = (
    "The following are survey responses from students in their senior year at Dartmouth College.\n"
    f"The prompt was: '{question}'\n"
    "Identify the topics mentioned in each response, "
    "as well as the sentiments expressed towards those topics. "
    "Format your response in valid JSON. Here is an example: \n\n"
    f"Response: \n'{example_response}'\n\n"
    f"Output: \n{example_output}\n\n"
    "Here are the responses to process, separated by '\\n------\\n':\n\n"
    f"{'\n------\n'.join(responses)}"
)

print(prompt[:1500])

The following are survey responses from students in their senior year at Dartmouth College.
The prompt was: 'Please use the space below to describe the most important outcomes of your time as an undergraduate.'
Identify the topics mentioned in each response, as well as the sentiments expressed towards those topics. Format your response in valid JSON. Here is an example: 

Response: 
'Dartmouth has been a transformative experience that's shaped my academic, personal, and professional growth. I've gained invaluable knowledge and skills through rigorous courses in econ and international relations, which have helped me develop a nuanced perspective on global issues. However, I've also struggled with the intense academic pressure and lack of diversity in certain departments.'

Output: 
{
    "response" : <original response>,
    "topics": [
        {"name": "academic and personal growth", "sentiment": "positive"},
        {"name": "academic rigor", "sentiment": "positive"},
        {"name":

In [5]:
response = llm.invoke(prompt)

TypeError: Object of type FieldInfo is not JSON serializable

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

parser.invoke(response)